<a href="https://colab.research.google.com/github/VishwasMore/creaditcard-fraud-detection/blob/master/creditcard_fraud_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!unzip "drive/My Drive/creditcard-fraud-detection/P39-Credit-Card-Fraud.zip" -d "/content/drive/My Drive/creditcard-fraud-detection"

In [0]:
import pandas as pd